In [14]:
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import platform
import numpy as np

# 폰트 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False

In [16]:
import pandas as pd
import numpy as np

# CSV 파일 읽기
examinees_df = pd.read_csv('./datas/수험자_1000.csv')
questions_df = pd.read_csv('exam1.csv')

# 30명의 수험자 랜덤 추출
selected_examinees = examinees_df.sample(n=30, random_state=42)

# 문항유형 결측치 및 데이터 타입 처리
questions_df['문항유형'] = questions_df['문항유형'].fillna('객관식').astype(str)

# 응답 데이터 생성 함수
def generate_response(examinee, questions):
    num_questions = len(questions)
    korean_level = examinee['한국어 수준']
    
    # 한국어 수준 검증 및 기본값 설정
    if pd.isnull(korean_level) or korean_level < 1 or korean_level > 5:
        korean_level = 3  # 기본값 설정
    
    # 정규분포를 사용하여 점수 생성 (평균 70, 표준편차 20, 범위 0-100)
    score = np.clip(np.random.normal(70, 20), 0, 100)
    
    # 한국어 수준에 따른 정답률 조정
    base_correct_rate = score / 100
    correct_rate = base_correct_rate * (korean_level / 5)
    correct_rate = np.clip(correct_rate, 0, 1)
    
    # 확률값이 유효한지 확인
    if np.isnan(correct_rate) or correct_rate <= 0 or correct_rate >= 1:
        correct_rate = 0.5  # 기본값 설정
    
    p = [correct_rate, 1 - correct_rate]
    p = p / np.sum(p)  # 확률의 합을 1로 보정
    
    ox_list = np.random.choice(['O', 'X'], size=num_questions, p=p)
    
    selected_answers = []
    for q_type, ox in zip(questions['문항유형'], ox_list):
        if q_type == '객관식' and ox == 'O':
            selected_answers.append(np.random.randint(1, 5))  # 1~4 중 랜덤 선택
        else:
            selected_answers.append(0)
    
    result = 'PASS' if score >= 60 else 'FAIL'
    
    return {
        '외국인 등록번호': examinee['외국인 등록번호'],
        '이름': examinee['이름'],
        '성별': examinee['성별'],
        '나이': examinee['나이'],
        '국적': examinee['국적'],
        '거주지': examinee['거주지'],
        '직업': examinee['직업'],
        '한국어 수준': korean_level,
        'OX리스트': ''.join(ox_list),
        '선택답안': ','.join(map(str, selected_answers)),
        '점수': round(score, 2),
        '결과': result,
        '문항코드리스트': ','.join(map(str, questions['문항코드'].tolist()))
    }

# 30명의 응답 데이터 생성
responses = []
for _, examinee in selected_examinees.iterrows():
    try:
        response = generate_response(examinee, questions_df)
        responses.append(response)
    except Exception as e:
        print(f"수험자 {examinee['이름']}의 응답 생성 중 오류 발생: {e}")

# 응답이 생성되지 않은 수험자가 있는지 확인
if len(responses) < 30:
    print(f"{30 - len(responses)}명의 응답이 생성되지 않았습니다.")

# DataFrame으로 변환
response_df = pd.DataFrame(responses)

# 결과 출력 (첫 5개 행만)
print(response_df.head())

# CSV 파일로 저장
response_df.to_csv('응답_데이터.csv', index=False, encoding='utf-8-sig')
print("응답 데이터가 '응답_데이터.csv' 파일로 저장되었습니다.")


     외국인 등록번호         이름 성별  나이      국적 거주지    직업  한국어 수준  \
0  3490941780    Gulnara  남  36  우즈베키스탄  울산   노동자       2   
1  4870107052    Dilnoza  여  52  우즈베키스탄  광주   예술가       3   
2  8426465635        Ali  남  18  우즈베키스탄  인천   예술가       1   
3  7784311902  Narantuya  남  37      몽골  울산   예술가       2   
4  5452114233    Preecha  남  18      태국  대구  언어연수       4   

                                               OX리스트  \
0  XXXOXXXXXXXXXXXXOXXOXOXXOXOXOOOXOOXXXXOOXOOXXO...   
1  XOXXOOXXXXOXXXOXXOXXXOXOOXXXOXOOXXXXOXXOOXXXXX...   
2  XXXXXXXXOXXXXXXXXOXXXXXXXXXXXXXOOXOOXXOXXXXOXX...   
3  OXOXOXXOXXXXXOOOXOOXXOXOXOOXXXOOXOXXXOXOOOXXOO...   
4  XOXXXXOXOOXOXXOXOXXXXOXXXXXXXOOOXOXXXOXOXXXXXO...   

                                                선택답안     점수    결과  \
0  0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,2,0,...  99.41  PASS   
1  0,4,0,0,1,4,0,0,0,0,4,0,0,0,1,0,0,4,0,0,0,2,0,...  51.57  FAIL   
2  0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,3,0,0,0,0,0,...  90.59  PASS   
3  3,0,1,0,4,0,0,2,0

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc

# 한글 폰트 설정
font_path = "C:/Windows/Fonts/malgun.ttf"  # 윈도우의 맑은 고딕 폰트 경로
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

# 데이터 로드
response_df = pd.read_csv('응답_데이터.csv')
examinees_df = pd.read_csv('수험자_1000.csv')

# 데이터 병합
df = pd.merge(response_df, examinees_df, on=['외국인 등록번호', '이름'])

# 데이터 확인
print(df.head())
print(df.info())

# 1. 점수 분포 히스토그램
plt.figure(figsize=(10, 6))
sns.histplot(df['점수'], kde=True)
plt.title('점수 분포 히스토그램')
plt.xlabel('점수')
plt.ylabel('빈도')
plt.show()

# 2. 통과율 원형 차트
plt.figure(figsize=(8, 8))
pass_counts = df['결과'].value_counts()
plt.pie(pass_counts.values, labels=pass_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('결과 분포 원형 차트')
plt.show()

# 3. 한국어 수준별 평균 점수 막대 그래프
plt.figure(figsize=(10, 6))
sns.barplot(x='한국어 수준', y='점수', data=df)
plt.title('한국어 수준별 평균 점수')
plt.xlabel('한국어 수준')
plt.ylabel('평균 점수')
plt.show()

# 4. 문항별 정답률 선 그래프
ox_data = df['OX리스트'].apply(lambda x: pd.Series(list(x)))
correct_rates = (ox_data == 'O').mean()
plt.figure(figsize=(12, 6))
plt.plot(correct_rates.index, correct_rates.values, marker='o')
plt.title('문항별 정답률')
plt.xlabel('문항 번호')
plt.ylabel('정답률')
plt.show()

# 5. 국적별 평균 점수 박스 플롯
plt.figure(figsize=(12, 6))
sns.boxplot(x='국적', y='점수', data=df)
plt.title('국적별 점수 분포')
plt.xlabel('국적')
plt.ylabel('점수')
plt.xticks(rotation=45)
plt.show()

# 6. 나이와 점수의 산점도
plt.figure(figsize=(10, 6))
sns.scatterplot(x='나이', y='점수', data=df)
plt.title('나이와 점수의 관계')
plt.xlabel('나이')
plt.ylabel('점수')
plt.show()

# 7. 성별에 따른 점수 비교 바이올린 플롯
plt.figure(figsize=(10, 6))
sns.violinplot(x='성별', y='점수', data=df)
plt.title('성별에 따른 점수 분포')
plt.xlabel('성별')
plt.ylabel('점수')
plt.show()

# 8. 직업별 평균 점수 막대 그래프
plt.figure(figsize=(12, 6))
sns.barplot(x='직업', y='점수', data=df)
plt.title('직업별 평균 점수')
plt.xlabel('직업')
plt.ylabel('평균 점수')
plt.xticks(rotation=45)
plt.show()

# 9. 거주지별 통과율 막대 그래프
pass_rate = df.groupby('거주지')['결과'].apply(lambda x: (x == '통과').mean())
plt.figure(figsize=(12, 6))
pass_rate.plot(kind='bar')
plt.title('거주지별 통과율')
plt.xlabel('거주지')
plt.ylabel('통과율')
plt.xticks(rotation=45)
plt.show()

# 10. 문항 유형별 정답률 (데이터가 없으므로 스킵)

# 11. 한국어 수준과 통과율의 관계 (수정된 버전)
plt.figure(figsize=(10, 6))
pass_rate_by_level = df.groupby('한국어 수준')['결과'].apply(lambda x: (x == '통과').mean())
pass_rate_by_level.plot(kind='bar')
plt.title('한국어 수준과 통과율의 관계')
plt.xlabel('한국어 수준')
plt.ylabel('통과율')
plt.xticks(rotation=45)
plt.show()

# 12. 점수와 OX 패턴의 상관관계 매트릭스
ox_data = df['OX리스트'].apply(lambda x: pd.Series(list(x)))
ox_data = ox_data.replace({'O': 1, 'X': 0})
correlation_matrix = pd.concat([df['점수'], ox_data], axis=1).corr()
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('점수와 OX 패턴의 상관관계')
plt.show()

print("모든 시각화가 완료되었습니다.")